In [18]:
import random
import pandas as pd
import numpy as np
import math


#产生随机种群
def randpop(NP,L):
    seed = "01"
    pop2 = []
    for i in range(NP):
        chr = []
        for j in range(L):
            chr.append(random.choice(seed))
        salt = ''.join(chr)
        pop2.append(salt)
    #print(pop2)
    return pop2


#解码
def jiema(NP,pop2):
    pop10 = []
    for i in range(NP):
        x = pop2[i][:3]
        y = pop2[i][3:]
        #二进制转十进制
        x1 = int(x,2)
        y1 = int(y,2)
        #print(m)
        pop10.append((x1,y1))
    #print(pop10)
    return pop10


#计算这个染色体对应解的适应度，即目标函数值
def getfit(NP,pop10):
    fit = []
    for i in range(NP):
        x = pop10[i][0]
        y = pop10[i][1]
        z = math.pow(x,2) + math.pow(y,2)
        fit.append(z)
        #print(y)
    #print(fit)
    return fit


#轮盘赌选择
def xuanze(NP,pop2,pop10,fit):
    df = pd.DataFrame({'pop2':pop2,'pop10':pop10,'fit':fit})
    #print(df)
    sum1 = 0
    for i in range(NP):
        sum1 = sum1+df.loc[i].fit
    #print(sum1)
    p_list = []
    for i in range(NP):
        p1 = df.loc[i].fit/sum1
        p_list.append(p1)
    #print(p_list)
    #print(len(p_list))
    pp_list = []
    for i in range(NP):
        pp = 0
        for j in range(i+1):
            pp1 = p_list[j]
            #print(pp1)
            pp = pp+pp1    
        #print(pp)
        pp_list.append(pp)
    #print(pp_list)
    #print(len(pp_list))
    df['p'] = p_list
    df['pp'] = pp_list
    #print(df)
    xuanze = []
    for i in range(NP):
        rand = random.random()
        #print(rand)
        n = 0
        for j in range(NP):
            if rand<pp_list[j]:
                n = j
                break
        #print(n)
        m = df.loc[n].pop2
        #print(m)
        xuanze.append(m)
    #print(xuanze)
    df['new'] = xuanze
    #print(df)
    #返回新的01字符串
    return xuanze



#执行基于概率Pc的两点交叉操作,采用奇偶交叉，即第1个和第2个交叉，第3个和第4个交叉，依次进行
def jiaocha(NP,Pc,L,pop2):
    pop2_ji = pop2[::2]
    pop2_ou = pop2[1::2]
    #print(pop2_ji)
    #print(pop2_ou)
    for i in range(int(NP/2)):
        rand = random.random()
        #print(rand)
        if rand<Pc:                  #控制交叉的染色体总数
            r = random.randint(0,L-1)
            #print(r)     #随机生成要交叉的基因位置
            #复制一个新的再交换
            pji = pop2_ji[i][r:]
            
            pou = pop2_ou[i][r:]
            pop2_ji[i] = pop2_ji[i][0:r] + pou
            #print(pop2_ji[i])
            pop2_ou[i] = pop2_ou[i][0:r] + pji
            #print(pop2_ou[i])
    #print(pop2_ji)
    #print(pop2_ou)
    pop2_new = pop2_ji + pop2_ou
    #print(pop2_new)
    return pop2_new


#执行基于概率Pm的变异操作，即随机选择一个基因位置取反，若是0则变为1，若是1则变为0
def bianyi(NP,Pm,L,pop2_new):
    #print(pop2_new)
    for i in range(NP):
        rand = random.random()
        #print(rand)
        if rand<Pm:
            r = random.randint(0,L-1)
            #print(r)    #随机生成要变异的基因位置
            if pop2_new[i][r]=='0':
                p = list(pop2_new[i])
                p[0]='1'
                pop2_new[i] = "".join(p)
            else:
                p = list(pop2_new[i])
                p[0]='0'
                pop2_new[i] = "".join(p)
    #print(pop2_new)
    return pop2_new

    
if __name__ == "__main__":
    NP=20;          #种群数量，一般取10~200，即染色体数目，将定义域划分50个（不等距）
    L=6;           #二进制位串长度，即染色体上基因数目，和计算要求的精度有关，如
                    #精度10^-5，则2^19<10*10^5<2^20，至少需要20位
    Pc=0.8;         #交叉率，一般取0.25~1
    Pm=0.1;         #变异率，一般取0.001~0.1
    G=100;          #最大遗传代数，一般100~1000

    trace = []
    
    #随机产生种群01字符串
    pop2 = randpop(NP,L)
    #print("pop2:",pop2)
    for i in range(G):
        #解码
        pop10 = jiema(NP,pop2)
        #print("pop10:",pop10)
        #求适应度
        fit = getfit(NP,pop10)
        #print("fit:",fit)
        
        print("第"+str(i)+"次:")
        df = pd.DataFrame({'pop2':pop2,'pop10':pop10,'fit':fit})
        print(df)
        #选择
        pop2 = xuanze(NP,pop2,pop10,fit)
        print("pop2:",pop2)
        #交叉
        pop2_new = jiaocha(NP,Pc,L,pop2)
        #变异
        pop2 = bianyi(NP,Pm,L,pop2_new)
        print("pop2_new:",pop2)



第0次:
      pop2   pop10   fit
0   001100  (1, 4)  17.0
1   110011  (6, 3)  45.0
2   001101  (1, 5)  26.0
3   010111  (2, 7)  53.0
4   000101  (0, 5)  25.0
5   100010  (4, 2)  20.0
6   110001  (6, 1)  37.0
7   010011  (2, 3)  13.0
8   011001  (3, 1)  10.0
9   100010  (4, 2)  20.0
10  100100  (4, 4)  32.0
11  010100  (2, 4)  20.0
12  101011  (5, 3)  34.0
13  111101  (7, 5)  74.0
14  001011  (1, 3)  10.0
15  110100  (6, 4)  52.0
16  010010  (2, 2)   8.0
17  101100  (5, 4)  41.0
18  100001  (4, 1)  17.0
19  000010  (0, 2)   4.0
pop2: ['100010', '110011', '101100', '001101', '000101', '111101', '001100', '101011', '101100', '100010', '010111', '010010', '111101', '110100', '100100', '101011', '110011', '100010', '100010', '010111']
pop2_new: ['100011', '101100', '011101', '001011', '101100', '010010', '110100', '101011', '110010', '110111', '010010', '001101', '100101', '101100', '100010', '010111', '111101', '100100', '100011', '100010']
第1次:
      pop2   pop10   fit
0   100011  (4, 3)  25

In [17]:
random.gauss(0.5,1)

1.3032827716893083